In [ ]:
import numpy as np
import pandas as pd

# Exploring covariance between properties at different scales
**Author**: Benjamin Jasperson

**GitHub URL**: https://github.com/bjasperson/property-covariance-demo

# Download Data
To begin, we need to import our project data.
Collecting this data ....

Reference [OpenKim](https://openkim.org/)

Download the CSV file and look at the first five rows.

In [ ]:
url_link = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/data.csv?raw=true"
df_data = pd.read_csv(url_link)
df_data.head()